In [1]:
import numpy as np
import cv2
import scipy
from scipy.spatial import distance as dist
import supervision as sv
import os
from tqdm import tqdm
from inference.models.yolo_world.yolo_world import YOLOWorld

model = YOLOWorld(model_id="yolo_world/l")
classes = ["person", "helmet",'Safety goggles','Safety Vest','Shoes',"Dog"]
model.set_classes(classes)
BOUNDING_BOX_ANNOTATOR = sv.BoundingBoxAnnotator(thickness=2)
LABEL_ANNOTATOR = sv.LabelAnnotator(text_thickness=2, text_scale=1, text_color=sv.Color.BLACK)

input_dir = "/Users/lebakuprathyushkumarreddy/Desktop/annotated_all/test/images"

output_dir = "/Users/lebakuprathyushkumarreddy/Desktop/testing_predicted"

for filename in os.listdir(input_dir):
    if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)
        results = model.infer(image, confidence=0.05)
        detections = sv.Detections.from_inference(results).with_nms(threshold=0.1)
        output_file = os.path.splitext(filename)[0] + "_detections.txt"
        output_path = os.path.join(output_dir, output_file)
        with open(output_path, "w") as file:
            for i in range(len(detections.xyxy)):
                class_id = detections.class_id[i]
                x_min = round(float(detections.xyxy[i][0]/640),7)
                y_min = round(float(detections.xyxy[i][1]/640),7)
                width = round(float((detections.xyxy[i][2] - detections.xyxy[i][0])/640),7)
                height =round(float((detections.xyxy[i][3] - detections.xyxy[i][1])/640),7)
                file.write(f"{class_id} {x_min} {y_min} {width} {height}\n")
                
        print(f"Detections saved to {output_path}")

#predictions saved 

[05/10/24 16:17:35] WARNING  Your inference package version 0.9.12rc1 is out of date! Please upgrade ]8;id=197781;file:///Users/lebakuprathyushkumarreddy/anaconda3/lib/python3.11/site-packages/inference/core/__init__.py\__init__.py]8;;\:]8;id=20559;file:///Users/lebakuprathyushkumarreddy/anaconda3/lib/python3.11/site-packages/inference/core/__init__.py#35\35]8;;\
                             to version 0.9.23 of inference for the latest features and bug fixes by               
                             running `pip install --upgrade inference`.                                            

In [144]:
import os
import numpy as np

def iou(box1, box2):
    x1, y1, x2, y2 = box1[:4]
    x3, y3, x4, y4 = box2[:4]
    xi1 = max(x1, x3)
    yi1 = max(y1, y3)
    xi2 = min(x2, x4)
    yi2 = min(y2, y4)
    intersection_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    union_area = box1_area + box2_area - intersection_area
    return intersection_area / union_area if union_area else 0

def read_annotation_file(file_path):

    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:5])
            xmin = x_center - width / 2
            ymin = y_center - height / 2
            xmax = x_center + width / 2
            ymax = y_center + height / 2
            boxes.append((xmin, ymin, xmax, ymax, class_id))
    return boxes

def find_matching_file(gt_file, predicted_files):

    base_name = gt_file.split('.')[0]
    for pred_file in predicted_files:
        if base_name in pred_file:
            return pred_file
    return None

def calculate_metrics(ground_truth_dir, predicted_dir):
    gt_files = [f for f in os.listdir(ground_truth_dir) if f.endswith('.txt')]
    pred_files = os.listdir(predicted_dir)
    class_stats = {}

    for gt_file in gt_files:
        gt_path = os.path.join(ground_truth_dir, gt_file)
        pred_file = find_matching_file(gt_file, pred_files)
        if pred_file:
            pred_path = os.path.join(predicted_dir, pred_file)
            gt_boxes = read_annotation_file(gt_path)
            pred_boxes = read_annotation_file(pred_path)

            for gt_box in gt_boxes:
                class_id = gt_box[4]
                if class_id not in class_stats:
                    class_stats[class_id] = {'tp': 0, 'fp': 0, 'fn': 0, 'ious': []}
                best_iou = 0
                best_pred_box = None

                for pred_box in pred_boxes:
                    if pred_box[4] == class_id:
                        current_iou = iou(gt_box, pred_box)
                        if current_iou > best_iou:
                            best_iou = current_iou
                            best_pred_box = pred_box

                if best_iou >= 0.1:
                    class_stats[class_id]['tp'] += 1
                    class_stats[class_id]['ious'].append(best_iou)
                else:
                    class_stats[class_id]['fn'] += 1

            for pred_box in pred_boxes:
                class_id = pred_box[4]
                if class_id not in class_stats:
                    class_stats[class_id] = {'tp': 0, 'fp': 0, 'fn': 0, 'ious': []}
                if not any(iou(pred_box, gt) >= 0.1 and gt[4] == class_id for gt in gt_boxes):
                    class_stats[class_id]['fp'] += 1

    for class_id, stats in class_stats.items():
        tp = stats['tp']
        fp = stats['fp']
        fn = stats['fn']
        ious = stats['ious']
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        mean_iou = np.mean(ious) if ious else 0
        print(f"Class {class_id} - Precision: {precision}, Recall: {recall}, Mean IOU: {mean_iou}")
        print()

ground_truth_dir = '/Users/lebakuprathyushkumarreddy/Desktop/annotated_all/test/labels'
predicted_dir = '/Users/lebakuprathyushkumarreddy/Desktop/testing_predicted'
calculate_metrics(ground_truth_dir, predicted_dir)

# classes = ['goggles--0', 'hammer--1', 'helmet--2', 'person--3', 'safetyvest--4']

Class 3 - Precision: 0.7928571428571428, Recall: 0.9736842105263158, Mean IOU: 0.1384478853798764

Class 2 - Precision: 0.7608695652173914, Recall: 0.7526881720430108, Mean IOU: 0.15836440187733375

Class 4 - Precision: 1.0, Recall: 0.008771929824561403, Mean IOU: 0.1538807167962647

Class 1 - Precision: 0.7142857142857143, Recall: 0.15151515151515152, Mean IOU: 0.1501783932140776

Class 0 - Precision: 1.0, Recall: 0.045454545454545456, Mean IOU: 0.1560808684441639

